# QA over Documents with Azure Cognitive Search as Vector Database

In [5]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.agents import initialize_agent, Tool, AgentType


load_dotenv(verbose=True)

OPENAI_API_TYPE = os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")

In [2]:
embeddings = OpenAIEmbeddings(deployment='text-embedding-ada-002')

vector_store = AzureSearch(
    azure_search_endpoint=os.getenv("VECTOR_STORE_ADDRESS"),
    azure_search_key=os.getenv("VECTOR_STORE_KEY"),
    index_name="mackenzie",
    embedding_function=embeddings.embed_query,
)

In [3]:
loader = PyPDFDirectoryLoader("pdfs")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs);

In [6]:
llm = AzureChatOpenAI(
    openai_api_base=OPENAI_API_BASE,
    openai_api_version=OPENAI_API_VERSION,
    deployment_name='gpt-35-turbo-16k',
    openai_api_key=OPENAI_API_KEY,
    openai_api_type=OPENAI_API_TYPE,
)

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(),
    return_source_documents=False
)

In [7]:
query = "What does a product manager do?"
result = qa({"query": query})

In [8]:
result

{'query': 'What does a product manager do?',
 'result': "A product manager is responsible for overseeing the development and management of a company's products. They work closely with cross-functional teams such as engineering, marketing, and sales to ensure that products align with the company's goals and meet customer needs. Some specific responsibilities of a product manager include:\n\n1. Developing product strategies and plans\n2. Conducting market research and identifying customer needs\n3. Creating and managing product roadmaps\n4. Collaborating with internal teams to ensure successful product execution\n5. Analyzing product performance and customer feedback to identify areas for improvement\n6. Overseeing product lifecycle management processes\n7. Developing pricing strategies and cost models\n8. Managing product budgets\n9. Leading and guiding the product management team."}

# OpenAI Functions Agent

In [10]:
from pydantic import BaseModel, Field
from langchain.tools.base import BaseTool
from typing import Type

class FakeUserInput(BaseModel):
    """Input for FakeUserTool."""

    gender: str = Field(..., description="Fake user gender. Available genders: 'male' or ''female")
    naturality: str = Field(..., description="Country ISO 3166 code")

class FakeUserTool(BaseTool):
    """Tool that creates fake user data."""

    name: str = "fake_user_tool"
    args_schema: Type[BaseModel] = FakeUserInput
    description: str = "Tool to generate fake user data considering a gender and a nationality."

    def _run(self, gender: str, naturality: str) -> str:
        try:
            import requests

            response = requests.get(
                'https://randomuser.me/api/1.4/', 
                params={'gender': gender,
                        'nat': naturality}
            )
            return response.text
        except Exception as e:
            return "Error: " + str(e)

In [11]:
tools = [FakeUserTool()]

In [12]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [13]:
agent('Create fake data for a male that was born in brazil')



> Entering new AgentExecutor chain...

Invoking: `fake_user_tool` with `{'gender': 'male', 'naturality': 'BR'}`


{"results":[{"gender":"male","name":{"title":"Mr","first":"Liciniano","last":"Mendes"},"location":{"street":{"number":435,"name":"Rua Santa Rita "},"city":"Nova Friburgo","state":"São Paulo","country":"Brazil","postcode":11839,"coordinates":{"latitude":"-9.5506","longitude":"36.1872"},"timezone":{"offset":"-3:30","description":"Newfoundland"}},"email":"liciniano.mendes@example.com","login":{"uuid":"940a9e01-fe9a-42d3-950e-07b6708ca42d","username":"greenbear921","password":"ariana","salt":"g76JOkZl","md5":"8a44c46ce248e1b18e27ba6bf4969e08","sha1":"db48f306c0bef54ebdf9e0a57112111062cd482e","sha256":"fea7afc7b247052687e14aa63b423f04e64ea3e286a52c6dfc25f8380718a441"},"dob":{"date":"1947-02-09T18:25:07.787Z","age":76},"registered":{"date":"2011-08-01T17:11:26.009Z","age":11},"phone":"(34) 0576-2183","cell":"(07) 6464-6132","id":{"name":"CPF","value":"405.332.070-09"},"picture"

{'input': 'Create fake data for a male that was born in brazil',
 'output': 'Sure, here is the generated data:\n\n- Full Name: Mr Liciniano Mendes\n- Location: Rua Santa Rita, 435, Nova Friburgo, São Paulo, Brazil, 11839\n- Coordinates: Latitude -9.5506, Longitude 36.1872\n- Timezone: Newfoundland (-3:30)\n- Email: liciniano.mendes@example.com\n- Username: greenbear921\n- Date of Birth: 1947-02-09 (Age: 76)\n- Registered since: 2011-08-01 (Registered for 11 years)\n- Phone: (34) 0576-2183\n- Cell: (07) 6464-6132\n- ID: CPF: 405.332.070-09\n- Picture: [Link](https://randomuser.me/api/portraits/men/70.jpg)\n- Nationality: BR\n\nPlease note that this information is fake and generated for the purpose of your request.'}

# OpenAI Functions Multi Tool Agent

In [14]:
qa_tool = Tool(
    name='role-library-tool',
    func=qa.run,
    description='Useful when a user wants answers about the company employee handbook'
)

tools = [FakeUserTool(), qa_tool]

In [15]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [16]:
agent('Create fake data for a female that was born in france')



> Entering new AgentExecutor chain...

Invoking: `fake_user_tool` with `{'gender': 'female', 'naturality': 'FR'}`


{"results":[{"gender":"female","name":{"title":"Mrs","first":"Emy","last":"Philippe"},"location":{"street":{"number":1945,"name":"Avenue du Fort-Caire"},"city":"Brest","state":"Alpes-de-Haute-Provence","country":"France","postcode":74064,"coordinates":{"latitude":"53.4326","longitude":"-53.8271"},"timezone":{"offset":"-7:00","description":"Mountain Time (US & Canada)"}},"email":"emy.philippe@example.com","login":{"uuid":"c833130b-28e0-4b43-9854-7b0aa4195ec2","username":"goldenfish535","password":"kappa","salt":"jF5dTpmu","md5":"c31e4939d9061cf9687f205f0284a041","sha1":"798a7f3a5efe67e47ad0e49ebf010961eb640666","sha256":"c3badea07d5a4c7c747855644d7625a25b3eddeabe5bcfca5cd8ae746a091546"},"dob":{"date":"1984-07-12T00:29:21.500Z","age":39},"registered":{"date":"2003-05-14T23:51:50.827Z","age":20},"phone":"04-20-63-87-12","cell":"06-65-67-82-12","id":{"name":"INSEE","value":

{'input': 'Create fake data for a female that was born in france',
 'output': 'Here is the generated fake user data:\n\n- Name: Mrs. Emy Philippe\n- Location: 1945 Avenue du Fort-Caire, Brest, Alpes-de-Haute-Provence, France\n- Postcode: 74064\n- Email: emy.philippe@example.com\n- Username: goldenfish535\n- Date of Birth: 1984-07-12\n- Registered since: 2003-05-14\n- Phone: 04-20-63-87-12\n- Cell: 06-65-67-82-12\n- ID: INSEE 2840629313726 16\n\n![Emy Philippe](https://randomuser.me/api/portraits/women/64.jpg)'}

In [ ]:
agent('What does a product manager do?')